In [9]:
import cv2
import os
from pathlib import Path

In [15]:
# 
path = Path("Dataset_TP3") 
videos_paths  = [p for p in path.iterdir() if p.is_file()]

videos = []
for file in videos_paths:
    vid = cv2.VideoCapture(file)
    if not vid.isOpened():
        print("Error: Could not open video.")
        continue
    videos.append(vid)